## Работа с Midjourney v6 

и другими моделями с помощью TT API

документация: https://ttapi.io/docs

In [ ]:
import os
import requests

from io import BytesIO
from PIL import Image

import time

from tqdm.notebook import tqdm

register here https://ttapi.io

get your api key here https://ttapi.io/center

In [ ]:
TT_API_KEY = ...

### sending generation requests

In [ ]:
endpoint_imagine = 'https://api.ttapi.io/midjourney/v1/imagine'
headers = {'TT-API-KEY': TT_API_KEY}

In [ ]:
PROMPT = '''satellite image of a desert with an unexcavated archeological site
view directly from above, as if taken from a plane
the image should cover approximately 80m * 80m
--ar 1:1 --v 6.1'''

In [ ]:
data_imagine = {
    'prompt': PROMPT,
    'model': 'relax',
    'hookUrl': '',
    'timeout': 300
}

In [ ]:
job_ids = {}

In [ ]:
total_n_requests = 400
for query_idx_start in tqdm(range(1, total_n_requests, 10)):
    for query_idx in range(query_idx_start, query_idx_start + 10):
        response_imagine = requests.post(endpoint_imagine, headers=headers, json=data_imagine)
        assert response_imagine.status_code == 200, response_imagine.json()
        job_id = response_imagine.json()['data']['jobId']
        job_ids[query_idx] = job_id
    time.sleep(60 * 12)

In [ ]:
with open('job_ids_final.txt', 'w') as f:
    for (query_idx, job_id) in job_ids.items():
        f.write(f'{query_idx}\t{job_id}\n')

### fetching generation results

on midjourney dashboard https://ttapi.io/center/jobs 

wait for status = success for all images

In [ ]:
endpoint_fetch = 'https://api.ttapi.io/midjourney/v1/fetch'
headers = {'TT-API-KEY': TT_API_KEY}

In [ ]:
job_ids = {}

with open('job_ids_final.txt', 'r') as f:
    for line in f.readlines():
        query_idx, job_id = line.split()
        job_ids[int(query_idx)] = job_id.strip()

In [ ]:
for query_idx in tqdm(range(1, 401)):

    data_fetch = {'jobId': job_ids[query_idx]}
    response_fetch = requests.post(endpoint_fetch, headers=headers, json=data_fetch)
    assert response_fetch.status_code == 200, response_fetch.json()

    img_url = response_fetch.json()['data']['cdnImage']
    img = Image.open(BytesIO(requests.get(img_url).content))

    img1 = img.crop((0, 0, 1024, 1024))
    img2 = img.crop((0, 1024, 1024, 2048))
    img3 = img.crop((1024, 0, 2048, 1024))
    img4 = img.crop((1024, 1024, 2048, 2048))

    img1.save(f'generated_midjourney/img{query_idx}_{1}.jpg')
    img2.save(f'generated_midjourney/img{query_idx}_{2}.jpg')
    img3.save(f'generated_midjourney/img{query_idx}_{3}.jpg')
    img4.save(f'generated_midjourney/img{query_idx}_{4}.jpg')

In [ ]:
len(os.listdir('generated_midjourney'))